# Introduction to the Monte Carlo method

----

Start by defining the [Gibbs (or Boltzmann) distribution](https://en.wikipedia.org/wiki/Boltzmann_distribution):
$$P(\alpha) = e^{-E(\alpha)/kT}$$
this expression, defines the probability of observing a particular configuration of spins, $\alpha$. 
As you can see, the probability of $\alpha$ decays exponentially with increasing energy of $\alpha$, $E(\alpha)$,
where $k$ is the Boltzmann constant, $k = 1.38064852 \times 10^{-23} J/K$
and $T$ is the temperature in Kelvin. 

## What defines the energy of a configuration of spins? 
Given a configuration of spins (e.g., $\uparrow\downarrow\downarrow\uparrow\downarrow$) we can define the energy using what is referred to as an Ising Hamiltonian:
$$ \hat{H}' = \frac{\hat{H}}{k} = -\frac{J}{k}\sum_{<ij>} s_is_j + \frac{\mu}{k}\sum_{i} S_i,$$
where, $s_i=1$ if the $i^{th}$ spin is `up` and $s_i=-1$ if it is `down`, and the brackets $<ij>$ indicate a sum over spins that are connected,
and $J$ is a constant that determines the energy scale. 
The energy here has been divided by the Boltzmann constant to yield units of temperature. 
Let's consider the following case, which has the sites connected in a single 1D line:
$$\alpha = \uparrow-\downarrow-\downarrow-\uparrow-\downarrow.$$ 
What is the energy of such a configuration?
$$ E(\alpha)' = \frac{-J}{k}(-1 + 1 - 1 - 1) + \frac{\mu}{k}(1 + -1 + -1 + 1 + -1) = \frac{E(\alpha)}{k} = 2\frac{J}{k} - \frac{\mu}{k}$$

## P1: Write a class that defines a spin configuration

In [23]:
class SpinConfig:
    def __init__(self, spin_str, closed=False):
        self.spin_str = [-1 if x == '-' else 1 for x in spin_str]
        self.index = 0
        self.boundary = closed
    def __repr__(self):
        return ''.join(['↓' if x == -1 else '↑' for x in self.spin_str])
    def __iter__(self):
        return self
    def __next__(self):
        try:
            res = self.spin_str[self.index]
        except IndexError:
            raise StopIteration
        self.index += 1
        return res
    def __getitem__(self, item):
         return self.spin_str[item]
    def __len__(self):
        return len(self.spin_str)
    def closed(self):
        return self.boundary

## P2: Write a class that defines the 1D hamiltonian, containing a function that computes the energy of a configuration

In [28]:
class Hamiltonian:
    def __init__(self, k, J, mu):
        self.k = k
        self.J = J
        self.mu = mu
    def energy(self, spin_config):
        n = len(spin_config)
        return (-self.J/self.k) * sum(spin_config[i] * spin_config[(i + 1) % n] for i in range(len(spin_config) - (0 if spin_config.closed() else 1))) + (self.mu/self.k) * sum(spin_config) 

## Properties
For any fixed state, $\alpha$, the `magnetization` ($M$) is proportional to the _excess_ number of spins pointing up or down while the energy is given by the
Hamiltonian:
$$M(\alpha) = N_{\text{up}}(\alpha) - N_{\text{down}}(\alpha).$$
As a dynamical, fluctuating system, each time you measure the magnetization, the system might be in a different state ($\alpha$) and so you'll get a different number!
However, we already know what the probability of measuring any particular $\alpha$ is, so in order to compute the average magnetization, $\left<M\right>$, we just need to multiply the magnetization of each possible configuration times the probability of it being measured, and then add them all up!
$$ \left<M\right> = \sum_\alpha M(\alpha)P(\alpha).$$
In fact, any average value can be obtained by adding up the value of an individual configuration multiplied by it's probability:
$$ \left<E\right> = \sum_\alpha E(\alpha)P(\alpha).$$

This means that to obtain any average value (also known as an `expectation value`) computationally, we must compute the both the value and probability of all possible configurations. This becomes extremely expensive as the number of spins ($N$) increases. 

## P3: Write a function that computes the magnetization of a spin configuration

In [17]:
magnetization = lambda spin_config : sum(spin_config)

## Q2: How many configurations are possible for:

(a) N=10?

In [5]:
2**10

1024

(b) N=100?

In [6]:
2**100

1267650600228229401496703205376

(c) N=1000?

In [7]:
2**1000

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376

## Q3: What is the energy for (++-+---+--+)?

In [29]:
H = Hamiltonian(1, -1, .1)
H.energy(SpinConfig("++-+---+--+"))

-2.1